### Temperature Model

####  Model Equation 
$R =R_{max}*\left(\frac{T_{max}-Temp}{T_{max}-T_{opt}}\right)*\left(\frac{Temp-T_{min}}{T_{opt}-T_{min}}\right)^\frac{T_{opt}-T_{min}}{T_{max}-T_{opt}}$

#### Parameters
$R$: Growth Rate  
$R_{max}$: Maximal Growth Rate (4.0)  
$T_{max}$: Maximal Growth Temperature (30)  
$T_{min}$: Minimal Growth Temperature (-5)       
$T_{opt}$: Optimal Growth Temperature (17.6)  

#### Required Input
Monthly maximal temperature

### Stream Flow Model

#### Chinook
$SpawnArea = \frac{a}{1+\left(\frac{Flow-F_{o}}{b}\right)^2}$  
$F_{o}: 424$  
$a: 12818$  
$b: 424$  

#### Sockeye
$SpawnArea = \frac{a}{1+\left(\frac{Flow-F_{o}}{b}\right)^2}$  
$F_{o}: 390$  
$a: 946$     
$b: 390$  

#### Coho
$SpawnArea = \frac{a}{1+\left(\frac{Flow-F_{o}}{b}\right)^2}$  
Fo = 362   
a = 12124   
b = 362  

#### Pink
$SpawnArea =A_{max}*\left(\frac{F_{max}-Flow}{F_{max}-F_{opt}}\right)*\left(\frac{Flow-F_{min}}{F_{opt}-F_{min}}\right)^\frac{F_{opt}-F_{min}}{F_{max}-F_{opt}}$  
$A_{max}: 1052$    
$F_{max}: 1200$    
$F_{min}: 0$     
$F_{opt}: 420$  

### Model Run and Output Interpretation

#### Reading in future projections of stream temperature and stream flow

In [1]:
import fishmod_utils
import pandas as pd

In [2]:
data_wea = pd.read_csv("sites_streamflow_stream_temperature.csv")
data_latlon = pd.read_csv("full_site_test_dataset.csv")
data_latlon = data_latlon.iloc[:,0:4]

In [3]:
data_wea.head()

,Site ID,Stream Temperature 2040s,Stream Temperature 2080s,Streamflow 2040s,Streamflow 2080s
0,ALB,24.304,25.621,2042.155229,1955.022943
1,LIB,18.852,19.938,2727.771443,2488.091671
2,BFE,12.079,12.824,4217.675486,3687.116386
3,BON,19.896,21.094,67905.510514,70655.700086
4,HOD,22.852,23.982,251.917157,250.765671


In [4]:
data_wea40 = data_wea.ix[:,["Site ID", "Stream Temperature 2040s", "Streamflow 2040s"]]
data_wea80 = data_wea.ix[:,["Site ID", "Stream Temperature 2080s", "Streamflow 2080s"]]

In [5]:
data_wea40 = data_wea40.assign(Decade='2030-2059')
data_wea40.head()

,Site ID,Stream Temperature 2040s,Streamflow 2040s,Decade
0,ALB,24.304,2042.155229,2030-2059
1,LIB,18.852,2727.771443,2030-2059
2,BFE,12.079,4217.675486,2030-2059
3,BON,19.896,67905.510514,2030-2059
4,HOD,22.852,251.917157,2030-2059


In [6]:
data_wea80 = data_wea80.assign(Decade='2070-2099')
data_wea80.head()

,Site ID,Stream Temperature 2080s,Streamflow 2080s,Decade
0,ALB,25.621,1955.022943,2070-2099
1,LIB,19.938,2488.091671,2070-2099
2,BFE,12.824,3687.116386,2070-2099
3,BON,21.094,70655.700086,2070-2099
4,HOD,23.982,250.765671,2070-2099


In [7]:
data_wea40.columns = ["Site ID", "Temperature", "Streamflow", "Decade"]
data_wea80.columns = ["Site ID", "Temperature", "Streamflow", "Decade"]

In [8]:
data_wea40.shape

(347, 4)

In [9]:
data_wea40 = pd.merge(data_latlon, data_wea40)
data_wea40.head()

,Site ID,Basin,Latitude,Longitude,Temperature,Streamflow,Decade
0,ALB,Willamette,44.633300,-123.100000,24.304,2042.155229,2030-2059
1,LIB,Kootenay,48.411700,-115.309400,18.852,2727.771443,2030-2059
2,BFE,Libby,48.698056,-116.312500,12.079,4217.675486,2030-2059
3,BON,Lower Columbia,45.633333,-121.954444,19.896,67905.510514,2030-2059
4,HOD,Lower Columbia,45.655556,-121.547222,22.852,251.917157,2030-2059


In [10]:
data_wea80 = pd.merge(data_latlon, data_wea80)
data_wea80.head()

,Site ID,Basin,Latitude,Longitude,Temperature,Streamflow,Decade
0,ALB,Willamette,44.633300,-123.100000,25.621,1955.022943,2070-2099
1,LIB,Kootenay,48.411700,-115.309400,19.938,2488.091671,2070-2099
2,BFE,Libby,48.698056,-116.312500,12.824,3687.116386,2070-2099
3,BON,Lower Columbia,45.633333,-121.954444,21.094,70655.700086,2070-2099
4,HOD,Lower Columbia,45.655556,-121.547222,23.982,250.765671,2070-2099


#### Feeding futrue stream temperature into temperature model and translating outputs into fish happiness scale 1-5

In [11]:
# Stream temperature 2040s
temp = data_wea40["Temperature"]
temp_vulnerability40 = fishmod_utils.temp_mod(temp, r_max=4, t_max=30, t_min=-5, t_opt=17.6)

model output = temp_vulnerability


In [12]:
# Stream temperature 2080s
temp = data_wea80["Temperature"]
temp_vulnerability80 = fishmod_utils.temp_mod(temp, r_max=4, t_max=30, t_min=-5, t_opt=17.6)

model output = temp_vulnerability


#### Feeding futrue streamflow into streamflow model and translating outputs into fish happiness scale 1-5

In [13]:
## Stream flow 2040s
flow = data_wea40["Streamflow"]
# Chinook
flow_vulnerability40_chinook = fishmod_utils.stream_mod(flow, fo=424, a=12818, b=424)
# Sockeye
flow_vulnerability40_sockeye = fishmod_utils.stream_mod(flow, fo=390, a=946, b=390)
# Coho
flow_vulnerability40_coho = fishmod_utils.stream_mod(flow, fo=362, a=12124, b=362)
# Pink
flow_vulnerability40_pink = fishmod_utils.stream_mod_pink(flow, a_max=1052, f_max=1200, f_min=0, f_opt=420)

model output = flow_vulnerability
model output = flow_vulnerability
model output = flow_vulnerability
model output = flow_vulnerability_pink


In [14]:
## stream flow 2080s
flow = data_wea80["Streamflow"]
# Chinook
flow_vulnerability80_chinook = fishmod_utils.stream_mod(flow, fo=424, a=12818, b=424)
# Sockeye
flow_vulnerability80_sockeye = fishmod_utils.stream_mod(flow, fo=390, a=946, b=390)
# Coho
flow_vulnerability80_coho = fishmod_utils.stream_mod(flow, fo=362, a=12124, b=362)
# Pink
flow_vulnerability80_pink = fishmod_utils.stream_mod_pink(flow, a_max=1052, f_max=1200, f_min=0, f_opt=420)

model output = flow_vulnerability
model output = flow_vulnerability
model output = flow_vulnerability
model output = flow_vulnerability_pink


#### Combining temperature & streamflow vulnerability projections, assigning equal weight to each output

In [15]:
## Year 2040s
# Chinook
chinook_vulnerability40 = pd.DataFrame(round(temp_vulnerability40*0.5 + flow_vulnerability40_chinook*0.5))
# Sockeye
sockeye_vulnerability40 = pd.DataFrame(round(temp_vulnerability40*0.5 + flow_vulnerability40_sockeye*0.5))
# Coho
coho_vulnerability40 = pd.DataFrame(round(temp_vulnerability40*0.5 + flow_vulnerability40_coho*0.5))
# Pink
pink_vulnerability40 = pd.DataFrame(round(temp_vulnerability40*0.5 + flow_vulnerability40_pink*0.5))

In [16]:
## Year 2080s
# Chinook
chinook_vulnerability80 = pd.DataFrame(round(temp_vulnerability80*0.5 + flow_vulnerability80_chinook*0.5))
# Sockeye
sockeye_vulnerability80 = pd.DataFrame(round(temp_vulnerability80*0.5 + flow_vulnerability80_sockeye*0.5))
# Coho
coho_vulnerability80 = pd.DataFrame(round(temp_vulnerability80*0.5 + flow_vulnerability80_coho*0.5))
# Pink
pink_vulnerability80 = pd.DataFrame(round(temp_vulnerability80*0.5 + flow_vulnerability80_pink*0.5))

#### Combining all model outputs to a dataframe

In [17]:
## make a sub-dataframe for each species - 40s
# Chinook
chinook_40 = data_wea40
chinook_40 = chinook_40.assign(Species='Chinook')
chinook_40["Viability"] = chinook_vulnerability40.astype(int)
# Sockeye
sockeye_40 = data_wea40
sockeye_40 = sockeye_40.assign(Species='Sockeye')
sockeye_40["Viability"] = sockeye_vulnerability40.astype(int)
# Coho
coho_40 = data_wea40
coho_40 = coho_40.assign(Species='Coho')
coho_40["Viability"] = coho_vulnerability40.astype(int)
# Pink
pink_40 = data_wea40
pink_40 = pink_40.assign(Species='Pink')
pink_40["Viability"] = pink_vulnerability40.astype(int)

In [18]:
## make a sub-dataframe for each species - 80s
# Chinook
chinook_80 = data_wea80
chinook_80 = chinook_80.assign(Species='Chinook')
chinook_80["Viability"] = chinook_vulnerability80.astype(int)
# Sockeye
sockeye_80 = data_wea80
sockeye_80 = sockeye_80.assign(Species='Sockeye')
sockeye_80["Viability"] = sockeye_vulnerability80.astype(int)
# Coho
coho_80 = data_wea80
coho_80 = coho_80.assign(Species='Coho')
coho_80["Viability"] = coho_vulnerability80.astype(int)
# Pink
pink_80 = data_wea80
pink_80 = pink_80.assign(Species='Pink')
pink_80["Viability"] = pink_vulnerability80.astype(int)

#### Merging 40s fish species output

In [19]:
fish_vulnerability_40 = chinook_40.append(sockeye_40).append(coho_40).append(pink_40)
fish_vulnerability_40.tail()

,Site ID,Basin,Latitude,Longitude,Temperature,Streamflow,Decade,Species,Viability
344,BLAGA,Little Blackfoot,46.519483,-112.793172,18.678,16.628886,2030-2059,Pink,4
345,SILSP,Silver Bow Creek,46.179497,-112.780561,18.824,4.115029,2030-2059,Pink,3
346,CLAGA,Clark Fork,46.208242,-112.767350,18.860,77.583186,2030-2059,Pink,4
347,CLALO,Clark Fork,46.397650,-112.742539,18.884,110.710457,2030-2059,Pink,4
348,BLADA,Blackfoot,42.815278,-111.506667,13.516,33.855129,2030-2059,Pink,4


In [20]:
fish_vulnerability_80 = chinook_80.append(sockeye_80).append(coho_80).append(pink_80)
fish_vulnerability_80.tail()

,Site ID,Basin,Latitude,Longitude,Temperature,Streamflow,Decade,Species,Viability
344,BLAGA,Little Blackfoot,46.519483,-112.793172,19.760,21.142771,2070-2099,Pink,4
345,SILSP,Silver Bow Creek,46.179497,-112.780561,19.907,4.832086,2070-2099,Pink,3
346,CLAGA,Clark Fork,46.208242,-112.767350,19.943,81.986157,2070-2099,Pink,4
347,CLALO,Clark Fork,46.397650,-112.742539,19.968,114.338129,2070-2099,Pink,4
348,BLADA,Blackfoot,42.815278,-111.506667,14.535,43.218700,2070-2099,Pink,4


In [21]:
fish_vulnerability = fish_vulnerability_40.append(fish_vulnerability_80)
fish_vulnerability.head()

,Site ID,Basin,Latitude,Longitude,Temperature,Streamflow,Decade,Species,Viability
0,ALB,Willamette,44.633300,-123.100000,24.304,2042.155229,2030-2059,Chinook,3
1,LIB,Kootenay,48.411700,-115.309400,18.852,2727.771443,2030-2059,Chinook,3
2,BFE,Libby,48.698056,-116.312500,12.079,4217.675486,2030-2059,Chinook,3
3,BON,Lower Columbia,45.633333,-121.954444,19.896,67905.510514,2030-2059,Chinook,3
4,HOD,Lower Columbia,45.655556,-121.547222,22.852,251.917157,2030-2059,Chinook,5


In [22]:
fish_vulnerability.to_csv("fish_vulnerability.csv")